# STEP 1: Imports and Get Data

In [14]:
from model_components.data_reader import FinancialTimeSeriesDataset
from model_components.transformers import TimeSeriesTransformer
from model_components.fin_trainer import TimeSeriesForecastingTrainer
from model_components.utils import (
    create_optimizer,
    create_scheduler,
    plot_lr_schedule
)
from model_components.synth_data import SyntheticTimeSeriesDataset
  
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import yaml
import gc
import torch
from torchinfo import summary
import os
import json
import tarfile
import shutil
import wandb
import yaml
from sklearn.preprocessing import MinMaxScaler, StandardScaler
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [15]:
%%writefile config.yaml

Name                      : "Nayesha-Gopal"

###### Dataset -----------------------------------------------------------------
data:                    # Currently setup for Colab assuming out setup
  root                 : "/local/hw4_data/hw4p1_data"  # TODO: Set the root path of your data
  train_partition      : "train"  # train
  val_partition        : "val"    # val
  test_partition       : "test"   # test
  subset               : 1.0      # Load a subset of the data (for debugging, testing, etc
  batch_size           : 16      #
  NUM_WORKERS          : 2        # Set to 0 for CPU
  forecast_horizon     : 1

###### Network Specs -------------------------------------------------------------
model: # Decoder-Only Language Model (HW4P1)
  d_model                   : 124
  d_ff                      : 64
  d_freq                    : 3    #how many dimensions to split time into- must be odd because it adds 1?
  num_layers                : 4
  num_heads                 : 16
  dropout                   : 0.2

###### Common Training Parameters ------------------------------------------------
training:
  use_wandb                   : False   # Toggle wandb logging
  wandb_run_id                : "none" # "none" or "run_id"
  resume                      : False  # Resume an existing run (run_id != 'none')
  epochs                      : 10
  gradient_accumulation_steps : 1
  wandb_project               : "IDL_final project PegasusNG" # wandb project to log to

###### Loss ----------------------------------------------------------------------
loss: # Just good ol' MSE
  label_smoothing: 0.0

###### Optimizer -----------------------------------------------------------------
optimizer:
  name: "adamw" # Options: sgd, adam, adamw
  lr: 5.0e-4   # Base learning rate

  # Common parameters
  weight_decay: 0.0001

  # Parameter groups
  param_groups:
    - name: self_attn
      patterns: []  # Will match all parameters containing keywords set their learning rate to 0.0001
      lr: 0.0001    # LR for self_attn
      layer_decay:
        enabled: False
        decay_rate: 0.8

    - name: ffn
      patterns: [] # Will match all parameters containing "ffn" and set their learning rate to 0.0001
      lr: 0.0001   # LR for ffn
      layer_decay:
        enabled: False
        decay_rate: 0.8

  # Layer-wise learning rates
  layer_decay:
    enabled: False
    decay_rate: 0.75

  # SGD specific parameters
  sgd:
    momentum: 0.9
    nesterov: True
    dampening: 0

  # Adam specific parameters
  adam:
    betas: [0.9, 0.999]
    eps: 1.0e-8
    amsgrad: False

  # AdamW specific parameters
  adamw:
    betas: [0.9, 0.999]
    eps: 1.0e-8
    amsgrad: False

###### Scheduler -----------------------------------------------------------------
scheduler:
  name: "cosine"  # Options: reduce_lr, cosine, cosine_warm

  # ReduceLROnPlateau specific parameters
  reduce_lr:
    mode: "min"  # Options: min, max
    factor: 0.1  # Factor to reduce learning rate by
    patience: 10  # Number of epochs with no improvement after which LR will be reduced
    threshold: 0.0001  # Threshold for measuring the new optimum
    threshold_mode: "rel"  # Options: rel, abs
    cooldown: 0  # Number of epochs to wait before resuming normal operation
    min_lr: 0.0000001  # Minimum learning rate
    eps: 1.0e-8  # Minimal decay applied to lr

  # CosineAnnealingLR specific parameters
  cosine:
    T_max: 15  # Maximum number of iterations
    eta_min: 1.0e-8  # Minimum learning rate
    last_epoch: -1

  # CosineAnnealingWarmRestarts specific parameters
  cosine_warm:
    T_0: 4  # Number of iterations for the first restart
    T_mult: 4  # Factor increasing T_i after each restart
    eta_min: 0.0000001  # Minimum learning rate
    last_epoch: -1

  # Warmup parameters (can be used with any scheduler)
  warmup:
    enabled: True
    type: "exponential"  # Options: linear, exponential
    epochs: 5
    start_factor: 0.1
    end_factor: 1.0

Overwriting config.yaml


In [16]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [17]:
# # create fake data for testing transformer
# # Create dataset
# train_dataset = SyntheticTimeSeriesDataset(
#     num_sequences=1000,
#     window_size=30,
#     forecast_horizon=5,
#     split='train',
#     noise_std=0.05,
#     num_frequencies=2
# )

# val_dataset = SyntheticTimeSeriesDataset(
#     num_sequences=1000,
#     window_size=30,
#     forecast_horizon=5,
#     split='val',
#     noise_std=0.05,
#     num_frequencies=2
# )

# # Sample visualization will auto-show
# print(train_dataset[0][0].shape)  # Should be [30, 1]
# print(train_dataset[0][1].shape)  # Should be [5, 1]

In [18]:
# CREATE DATASETS
train_dataset = FinancialTimeSeriesDataset(
    tickers=['AAPL'],
    start_date='2015-01-01',
    end_date='2024-12-31',
    window_size=5,
    forecast_horizon=config['data']['forecast_horizon'],
    target='Close',
    normalize='minmax',
    split='train',
    val_ratio=0.15,
    fit_scaler=True
)
tgt_scaler = train_dataset.tgt_scaler
cls_scaler = train_dataset.cls_scaler

val_dataset = FinancialTimeSeriesDataset(
    tickers=['AAPL'],
    start_date='2015-01-01',
    end_date='2024-12-31',
    window_size=5,
    forecast_horizon=config['data']['forecast_horizon'],
    target='Close',
    normalize='minmax',
    split='val',
    val_ratio=0.15,
    tgt_scaler=tgt_scaler,
    cls_scaler=cls_scaler
)




[*********************100%***********************]  1 of 1 completed

['Open', 'High', 'Low', 'Volume']
after normalization dataset is                 Open      High       Low    Volume
Date                                              
2015-01-02  0.017622  0.015956  0.014373  0.303048
2015-01-05  0.014716  0.013358  0.012551  0.373898
2015-01-06  0.013076  0.012221  0.011819  0.383564
2015-01-07  0.013695  0.012938  0.013763  0.219295
2015-01-08  0.015598  0.016617  0.015641  0.342434
after normalization dataset is Date
2015-01-02    0.015315
2015-01-05    0.012437
2015-01-06    0.012446
2015-01-07    0.013839
2015-01-08    0.017707
Name: Close, dtype: float64
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Time'], dtype='object')



Processing AAPL (train): 100%|██████████████████| 1066/1066 [00:00<00:00, 37170.81it/s]
[*********************100%***********************]  1 of 1 completed


['Open', 'High', 'Low', 'Volume']
after normalization dataset is                 Open      High       Low    Volume
Date                                              
2015-01-02  0.017622  0.015956  0.014373  0.303048
2015-01-05  0.014716  0.013358  0.012551  0.373898
2015-01-06  0.013076  0.012221  0.011819  0.383564
2015-01-07  0.013695  0.012938  0.013763  0.219295
2015-01-08  0.015598  0.016617  0.015641  0.342434
after normalization dataset is Date
2015-01-02    0.015315
2015-01-05    0.012437
2015-01-06    0.012446
2015-01-07    0.013839
2015-01-08    0.017707
Name: Close, dtype: float64
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Time'], dtype='object')


Processing AAPL (val): 100%|██████████████████████| 187/187 [00:00<00:00, 35099.56it/s]


In [19]:
# DATALOADERS
train_loader    = DataLoader(
    dataset     = train_dataset,
    batch_size  = config['data']['batch_size'],
    shuffle     = False,
    num_workers = config['data']['NUM_WORKERS'] if device == 'cuda' else 0,
    pin_memory  = True,
)

val_loader      = DataLoader(
    dataset     = val_dataset,
    batch_size  = config['data']['batch_size'],
    shuffle     = False,
    num_workers = config['data']['NUM_WORKERS'] if device == 'cuda' else 0,
    pin_memory  = True,
)

# STEP 2: Set Up Model

In [20]:
model_config = config['model']
# update this to add model input and output features
print(train_dataset.input_features, val_dataset.output_features)
model_config.update({
    'input_features': train_dataset.input_features,
    'output_features': val_dataset.output_features,              #hardcoded, TODO NEED TO FIX
    'forecast_horizon':  config['data']['forecast_horizon']
})
model = TimeSeriesTransformer(**model_config).to(device)

# CONFIRM EVERYTHING IS BEING LOADED CORRECTLY
for batch in train_loader:
    inputs, targets_shifted, targets_shifted_targfeat, targets_golden = batch

    print("First input sample (shape: {}):".format(inputs[0].shape))
    print(inputs[0])  # shape: [T, F]

    print("Corresponding target (shape: {}):".format(targets_shifted[0].shape))
    print(targets_shifted[0])  # shape: [forecast_steps]

    print("Inputs:", inputs.shape)
    print("Targets:", targets_shifted.shape)
    break

summary(model, input_data=(targets_shifted.to(device), targets_shifted_targfeat.to(device)))


6 1
First input sample (shape: torch.Size([5, 6])):
tensor([[0.0176, 0.0160, 0.0144, 0.0153, 0.3030, 0.0000],
        [0.0147, 0.0134, 0.0126, 0.0124, 0.3739, 1.0000],
        [0.0131, 0.0122, 0.0118, 0.0124, 0.3836, 2.0000],
        [0.0137, 0.0129, 0.0138, 0.0138, 0.2193, 3.0000],
        [0.0156, 0.0166, 0.0156, 0.0177, 0.3424, 4.0000]])
Corresponding target (shape: torch.Size([5, 6])):
tensor([[0.0176, 0.0160, 0.0144, 0.0153, 0.3030, 0.0000],
        [0.0176, 0.0160, 0.0144, 0.0153, 0.3030, 0.0000],
        [0.0147, 0.0134, 0.0126, 0.0124, 0.3739, 1.0000],
        [0.0131, 0.0122, 0.0118, 0.0124, 0.3836, 2.0000],
        [0.0137, 0.0129, 0.0138, 0.0138, 0.2193, 3.0000]])
Inputs: torch.Size([16, 5, 6])
Targets: torch.Size([16, 5, 6])
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])


Layer (type:depth-idx)                        Output Shape              Param #
TimeSeriesTransformer                         [16, 5, 1]                1
├─ModuleList: 1-1                             --                        --
│    └─Sequential: 2-1                        [16, 5, 124]              --
│    │    └─Linear: 3-1                       [16, 5, 124]              248
│    │    └─GELU: 3-2                         [16, 5, 124]              --
│    │    └─LayerNorm: 3-3                    [16, 5, 124]              248
│    │    └─Dropout: 3-4                      [16, 5, 124]              --
│    └─Sequential: 2-2                        [16, 5, 124]              --
│    │    └─Linear: 3-5                       [16, 5, 124]              248
│    │    └─GELU: 3-6                         [16, 5, 124]              --
│    │    └─LayerNorm: 3-7                    [16, 5, 124]              248
│    │    └─Dropout: 3-8                      [16, 5, 124]              --
│    └─Sequential

# STEP 3: Setup WandB and Trainer

In [21]:
wandb.login(key="b78dba2c54228a3e3e32bd91a1a1efca38d69bb9")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /jet/home/gandotra/.netrc


True

In [22]:
# wandb.finish()

In [23]:
trainer = TimeSeriesForecastingTrainer(
    model=model,
    config=config,
    run_name="test-1-fin2",
    config_file="config.yaml",
    scaler=tgt_scaler,
    device=device
)
print("done with init!")

False
done with init!


In [24]:
trainer.optimizer = create_optimizer(
    model=model,
    opt_config=config['optimizer']
)

trainer.scheduler = create_scheduler(
    optimizer=trainer.optimizer,
    scheduler_config=config['scheduler'],
    train_loader=train_loader,
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps']
)

plot_lr_schedule(
    scheduler=trainer.scheduler,
    num_epochs=config['training']['epochs'], 
    train_loader=train_loader,
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps']
)

# Initialize WandB
# if trainer.use_wandb:
#     wandb.init(
#         project=trainer.config['training'].get('wandb_project', 'time-series-forecasting'),
#         config=trainer.config,
#         name=trainer.run_name,
#         settings=wandb.Settings(
#         start_method="thread",             # don't spawn multiprocessing process
#         )
#     )


here!

🔧 Configuring Optimizer:
├── Type: ADAMW
├── Base LR: 0.0005
├── Weight Decay: 0.0001
├── Parameter Groups:
│   ├── Group: self_attn
│   │   ├── LR: 0.0001
│   │   └── Patterns: []
│   ├── Group: ffn
│   │   ├── LR: 0.0001
│   │   └── Patterns: []
│   └── Default Group (unmatched parameters)
└── AdamW Specific:
    ├── Betas: [0.9, 0.999]
    ├── Epsilon: 1e-08
    └── AMSGrad: False

📈 Configuring Learning Rate Scheduler:
├── Type: COSINE
├── Cosine Annealing Settings:
│   ├── T_max: 15 epochs (1005 steps)
│   └── Min LR: 1e-08
├── Warmup Settings:
│   ├── Duration: 5 epochs (335 steps)
│   ├── Start Factor: 0.1
│   └── End Factor: 1.0


/jet/home/psamal/hw_envs/idl_hw4/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


# Step 4: TRAIN!!   

In [25]:
print("here!")
trainer.train(train_loader, val_loader, epochs=50)

here!


Training:   0%|                                                 | 0/67 [00:00<?, ?it/s]/jet/home/psamal/hw_envs/idl_hw4/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MeanAbsoluteError was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
Training:   4%| | 3/67 [00:00<00:06,  9.86it/s, batch_accuracy=2.50%, loss=0.0068, mae=

shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.0014187104534357786
abm_loss is 0.004464433528482914
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.0019057843601331115
abm_loss is 0.004618532489985228
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.002661271020770073
abm_loss is 0.0033978417050093412
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.004176463466137648
abm_loss is 0.003004132304340601


Training:   7%| | 5/67 [00:00<00:02, 22.90it/s, batch_accuracy=1.25%, loss=0.0074, mae=

shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.0054927184246480465
abm_loss is 0.002702927915379405
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.00653843441978097
abm_loss is 0.003015449270606041


...

mse_loss is 0.8109022378921509
abm_loss is 0.49373698234558105
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.7406600117683411
abm_loss is 0.4364583492279053
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.9313387274742126
abm_loss is 0.3191041946411133
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.8813595771789551
abm_loss is 0.13044248521327972
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 1.0148134231567383
abm_loss is 0.24379120767116547
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.8276605606079102
abm_loss is 0.3270505368709564
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])


Training:  99%|▉| 66/67 [00:02<00:00, 29.61it/s, batch_accuracy=10.00%, loss=1.4379, ma

mse_loss is 0.6857713460922241
abm_loss is 0.2742774188518524
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.4777812063694
abm_loss is 0.3119651973247528
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.5476546287536621
abm_loss is 0.3745666444301605
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.5525646209716797
abm_loss is 0.5405416488647461
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.8689249157905579
abm_loss is 0.4150291979312897
shape of tgt_combined: torch.Size([10, 5, 128]), shape of memory_combined: torch.Size([10, 5, 128])
mse_loss is 0.8639443516731262
abm_loss is 0.638934850692749


Training: 100%|█| 67/67 [00:02<00:00, 28.61it/s, batch_accuracy=10.00%, loss=1.4379, ma



📈 Epoch 49 Metrics:
  TRAIN      | train_loss: 0.3344 | train_mae: nan | train_accuracy: 6.5291


In [26]:
print("here!")
trainer.evaluate(val_loader)

here!
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.015515211038291454
abm_loss is 2.539304494857788
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.017630144953727722
abm_loss is 2.169987440109253
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.027984825894236565
abm_loss is 2.062049627304077
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.09080802649259567
abm_loss is 2.1415748596191406
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.11213040351867676
abm_loss is 1.9197367429733276
shape of tgt_combined: torch.Size([16, 5, 128]), shape of memory_combined: torch.Size([16, 5, 128])
mse_loss is 0.09724217653274536
abm_loss is 1.6139999628067017
shape of tgt_com

{'val_loss': 2.808370962499935,
 'val_mae': 0.5506010055541992,
 'val_accuracy': 0.0}